# TOPSIS Implementation in Python

**TOPSIS** (Technique for Order of Preference by Similarity to Ideal Solution) is a multi-criteria decision analysis method.

---

## Author Information
- **Name:** Your Name
- **Roll No:** Your Roll Number

---

## 1. Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## 2. Create Sample Data

Creating a sample dataset with 8 alternatives (M1-M8) and 5 criteria (P1-P5).

In [ ]:
# Sample data
data = {
    'Fund Name': ['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8'],
    'P1': [0.67, 0.6, 0.82, 0.6, 0.76, 0.69, 0.79, 0.84],
    'P2': [0.45, 0.36, 0.67, 0.36, 0.58, 0.48, 0.62, 0.71],
    'P3': [6.5, 3.6, 3.8, 3.5, 4.8, 6.6, 4.8, 6.5],
    'P4': [42.6, 53.3, 63.1, 69.2, 43, 48.7, 59.2, 34.5],
    'P5': [12.56, 14.47, 17.1, 18.42, 12.29, 14.12, 16.35, 10.64]
}

df = pd.DataFrame(data)
print("Input Data:")
print(df.to_string(index=False))

In [ ]:
# Save to CSV (for command-line usage)
df.to_csv('data.csv', index=False)
print("Data saved to 'data.csv'")

## 3. Define Weights and Impacts

- **Weights:** Relative importance of each criterion
- **Impacts:** 
  - `+` (Positive/Benefit): Higher values are better
  - `-` (Negative/Cost): Lower values are better

In [ ]:
# Define weights for each criterion (P1, P2, P3, P4, P5)
weights = np.array([1, 1, 1, 2, 1])

# Define impacts: '+' for benefit criteria, '-' for cost criteria
impacts = ['+', '+', '-', '+', '-']

print(f"Weights: {weights}")
print(f"Impacts: {impacts}")

## 4. TOPSIS Implementation

### Step 4.1: Extract Decision Matrix

In [ ]:
# Extract the decision matrix (excluding the first column - Fund Names)
decision_matrix = df.iloc[:, 1:].values.astype(float)

print("Decision Matrix:")
print(decision_matrix)

### Step 4.2: Normalize the Decision Matrix

Using **Vector Normalization**:

$$r_{ij} = \frac{x_{ij}}{\sqrt{\sum_{i=1}^{m} x_{ij}^2}}$$

Where:
- $x_{ij}$ is the original value
- $r_{ij}$ is the normalized value

In [ ]:
def normalize_matrix(matrix):
    """
    Normalize the decision matrix using vector normalization.
    """
    # Calculate the square root of sum of squares for each column
    norm_factors = np.sqrt(np.sum(matrix ** 2, axis=0))
    
    # Normalize
    normalized = matrix / norm_factors
    
    return normalized

normalized_matrix = normalize_matrix(decision_matrix)

print("Normalized Decision Matrix:")
print(np.round(normalized_matrix, 4))

### Step 4.3: Calculate Weighted Normalized Matrix

$$v_{ij} = w_j \times r_{ij}$$

Where:
- $w_j$ is the weight of criterion $j$
- $v_{ij}$ is the weighted normalized value

In [ ]:
def calculate_weighted_normalized_matrix(normalized_matrix, weights):
    """
    Calculate the weighted normalized decision matrix.
    """
    return normalized_matrix * weights

weighted_matrix = calculate_weighted_normalized_matrix(normalized_matrix, weights)

print("Weighted Normalized Matrix:")
print(np.round(weighted_matrix, 4))

### Step 4.4: Determine Ideal Best and Ideal Worst Solutions

**Ideal Best (V+):**
- For benefit criteria (+): Maximum value in the column
- For cost criteria (-): Minimum value in the column

**Ideal Worst (V-):**
- For benefit criteria (+): Minimum value in the column
- For cost criteria (-): Maximum value in the column

In [ ]:
def find_ideal_solutions(weighted_matrix, impacts):
    """
    Find the ideal best (V+) and ideal worst (V-) solutions.
    """
    ideal_best = np.zeros(weighted_matrix.shape[1])
    ideal_worst = np.zeros(weighted_matrix.shape[1])
    
    for j, impact in enumerate(impacts):
        if impact == '+':
            # Benefit criteria: higher is better
            ideal_best[j] = np.max(weighted_matrix[:, j])
            ideal_worst[j] = np.min(weighted_matrix[:, j])
        else:
            # Cost criteria: lower is better
            ideal_best[j] = np.min(weighted_matrix[:, j])
            ideal_worst[j] = np.max(weighted_matrix[:, j])
    
    return ideal_best, ideal_worst

ideal_best, ideal_worst = find_ideal_solutions(weighted_matrix, impacts)

print("Ideal Best (V+):", np.round(ideal_best, 4))
print("Ideal Worst (V-):", np.round(ideal_worst, 4))

### Step 4.5: Calculate Separation Measures

**Distance from Ideal Best (S+):**

$$S_i^+ = \sqrt{\sum_{j=1}^{n} (v_{ij} - v_j^+)^2}$$

**Distance from Ideal Worst (S-):**

$$S_i^- = \sqrt{\sum_{j=1}^{n} (v_{ij} - v_j^-)^2}$$

In [ ]:
def calculate_separation_measures(weighted_matrix, ideal_best, ideal_worst):
    """
    Calculate separation measures from ideal best and ideal worst.
    """
    # Distance from ideal best (S+)
    s_plus = np.sqrt(np.sum((weighted_matrix - ideal_best) ** 2, axis=1))
    
    # Distance from ideal worst (S-)
    s_minus = np.sqrt(np.sum((weighted_matrix - ideal_worst) ** 2, axis=1))
    
    return s_plus, s_minus

s_plus, s_minus = calculate_separation_measures(weighted_matrix, ideal_best, ideal_worst)

print("Distance from Ideal Best (S+):", np.round(s_plus, 4))
print("Distance from Ideal Worst (S-):", np.round(s_minus, 4))

### Step 4.6: Calculate TOPSIS Score (Relative Closeness)

$$C_i = \frac{S_i^-}{S_i^+ + S_i^-}$$

Where:
- $C_i$ is the TOPSIS score for alternative $i$
- $0 \leq C_i \leq 1$
- Higher score indicates better alternative

In [ ]:
def calculate_topsis_score(s_plus, s_minus):
    """
    Calculate the TOPSIS score (relative closeness to ideal solution).
    """
    score = s_minus / (s_plus + s_minus)
    return score

scores = calculate_topsis_score(s_plus, s_minus)

print("TOPSIS Scores:", np.round(scores, 4))

### Step 4.7: Calculate Ranks

In [ ]:
def calculate_rank(scores):
    """
    Calculate ranks based on TOPSIS scores.
    Higher score = Better rank (Rank 1 is best)
    """
    ranks = scores.argsort()[::-1].argsort() + 1
    return ranks

ranks = calculate_rank(scores)

print("Ranks:", ranks)

## 5. Final Results

In [ ]:
# Add results to dataframe
df['Topsis Score'] = np.round(scores, 2)
df['Rank'] = ranks.astype(int)

print("\n" + "="*80)
print("FINAL RESULTS")
print("="*80)
print(df.to_string(index=False))

In [ ]:
# Save results to CSV
df.to_csv('output-result.csv', index=False)
print("\nResults saved to 'output-result.csv'")

## 6. Visualization

In [ ]:
# Set style
plt.style.use('seaborn-v0_8-whitegrid')
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: TOPSIS Scores Bar Chart
colors = plt.cm.RdYlGn(scores / scores.max())
bars = axes[0].bar(df['Fund Name'], df['Topsis Score'], color=colors, edgecolor='black')
axes[0].set_xlabel('Fund Name', fontsize=12)
axes[0].set_ylabel('TOPSIS Score', fontsize=12)
axes[0].set_title('TOPSIS Scores by Alternative', fontsize=14, fontweight='bold')
axes[0].set_ylim(0, 1)

# Add value labels on bars
for bar, score in zip(bars, df['Topsis Score']):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
                 f'{score:.2f}', ha='center', va='bottom', fontsize=10)

# Plot 2: Ranking Chart
sorted_df = df.sort_values('Rank')
colors2 = plt.cm.RdYlGn(np.linspace(0.9, 0.2, len(sorted_df)))
bars2 = axes[1].barh(sorted_df['Fund Name'], sorted_df['Topsis Score'], color=colors2, edgecolor='black')
axes[1].set_xlabel('TOPSIS Score', fontsize=12)
axes[1].set_ylabel('Fund Name', fontsize=12)
axes[1].set_title('Alternatives Ranked by TOPSIS Score', fontsize=14, fontweight='bold')
axes[1].set_xlim(0, 1)

# Add rank labels
for i, (bar, rank) in enumerate(zip(bars2, sorted_df['Rank'])):
    axes[1].text(bar.get_width() + 0.02, bar.get_y() + bar.get_height()/2,
                 f'Rank {rank}', ha='left', va='center', fontsize=10)

plt.tight_layout()
plt.savefig('topsis_results.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nGraph saved to 'topsis_results.png'")

## 7. Complete TOPSIS Function

A complete function that combines all steps for easy reuse.

In [ ]:
def topsis_complete(data, weights, impacts):
    """
    Complete TOPSIS implementation.
    
    Parameters:
    -----------
    data : pandas.DataFrame
        Input data with first column as alternative names and rest as criteria values
    weights : list or numpy.array
        Weights for each criterion
    impacts : list
        List of '+' or '-' indicating benefit or cost criteria
        
    Returns:
    --------
    pandas.DataFrame
        Original data with TOPSIS Score and Rank columns added
    """
    # Extract decision matrix
    decision_matrix = data.iloc[:, 1:].values.astype(float)
    weights = np.array(weights)
    
    # Step 1: Normalize
    norm_factors = np.sqrt(np.sum(decision_matrix ** 2, axis=0))
    normalized_matrix = decision_matrix / norm_factors
    
    # Step 2: Weighted normalized matrix
    weighted_matrix = normalized_matrix * weights
    
    # Step 3: Ideal best and worst
    ideal_best = np.zeros(weighted_matrix.shape[1])
    ideal_worst = np.zeros(weighted_matrix.shape[1])
    
    for j, impact in enumerate(impacts):
        if impact == '+':
            ideal_best[j] = np.max(weighted_matrix[:, j])
            ideal_worst[j] = np.min(weighted_matrix[:, j])
        else:
            ideal_best[j] = np.min(weighted_matrix[:, j])
            ideal_worst[j] = np.max(weighted_matrix[:, j])
    
    # Step 4: Separation measures
    s_plus = np.sqrt(np.sum((weighted_matrix - ideal_best) ** 2, axis=1))
    s_minus = np.sqrt(np.sum((weighted_matrix - ideal_worst) ** 2, axis=1))
    
    # Step 5: TOPSIS score
    scores = s_minus / (s_plus + s_minus)
    
    # Step 6: Rank
    ranks = scores.argsort()[::-1].argsort() + 1
    
    # Add to dataframe
    result = data.copy()
    result['Topsis Score'] = np.round(scores, 2)
    result['Rank'] = ranks.astype(int)
    
    return result

# Test the complete function
print("Testing complete TOPSIS function:")
print("="*80)

# Reload fresh data
test_df = pd.read_csv('data.csv')
test_weights = [1, 1, 1, 2, 1]
test_impacts = ['+', '+', '-', '+', '-']

result = topsis_complete(test_df, test_weights, test_impacts)
print(result.to_string(index=False))

## 8. Command Line Usage

The `topsis.py` script can be run from the command line:

```bash
python topsis.py data.csv "1,1,1,2,1" "+,+,-,+,-" output-result.csv
```